In [1]:
import numpy as np
import torch
from overrides import overrides
from typing import Iterable, Dict, List
import jsonlines
import warnings

from allennlp.models.archival import load_archive
# from allennlp.models.basic_classifier import BasicClassifier
# from allennlp.modules.seq2vec_encoders.cnn_encoder import CnnEncoder
# from allennlp.modules import FeedForward, Seq2SeqEncoder, Seq2VecEncoder, TextFieldEmbedder
from allennlp.predictors import Predictor #, TextClassifierPredictor

from allennlp.data import DatasetReader, Instance, TokenIndexer, Vocabulary #Tokenizer,
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers import Token #, Tokenizer, WordTokenizer
from allennlp.data.fields import MetadataField, TextField, LabelField

from allennlp.common.util import JsonDict




In [2]:
class ModalityDatasetReader(DatasetReader):
    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__()
        self._token_indexers = token_indexers or {'tokens': SingleIdTokenIndexer()}

    def _read(self, file_path: str) -> Iterable[Instance]:
        logger.info("Reading Modal Sense instances from {}".format(file_path))
        with open(file_path,"r") as file:
            for line in file:
                json_line = json.loads(line)
                json_line.pop("modal_verb", None)
                yield self.sentence_to_instance(**json_line)
    
    @overrides
    def text_to_instance(self, sentence:List[str], label:str=None) -> Instance:
        instance_dict = {"tokens": TextField(sentence, self._token_indexers)}
        
        if label is not None:
            instance_dict["label"] = LabelField(label)

        return Instance(instance_dict)

In [3]:
class ModalityPredictor(Predictor):

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        sentence = json_dict["tokens"]
        if not hasattr(self._dataset_reader, "tokenizer") and not hasattr(
            self._dataset_reader, "_tokenizer"
        ):
            sentence = [Token(t) for t in sentence.split()]            
        return self._dataset_reader.text_to_instance(sentence=sentence)

    
    def predict(self, sentence: str) -> JsonDict:
        return self.predict_json({"tokens": sentence})

In [6]:
def reset_mv_and_labels_counts(targets):
    modal_targets = {t: {"total": 0, "errors": 0} for t in targets}

    labels = {
        "dy": {"total": 0, "errors": 0},
        "de": {"total": 0, "errors": 0},
        "ep": {"total": 0, "errors": 0}
    }
    return modal_targets, labels

In [7]:
def collect_errors_and_accuarcy(filepath):
    labels_golds = {"de": [1,0,0], "ep": [0,1,0], "dy": [0,0,1]}
    with jsonlines.open(filepath, "r") as src:
        all_modal_targets = {line['modal_verb'].lower() for line in src}
        modal_verbs, labels = reset_mv_and_labels_counts(all_modal_targets)
    with jsonlines.open(filepath, "r") as test:
        predictions, golds = [], []
        for line in test:
            try:
                prediction = predictor.predict(sentence=line["sentence"])
                predicted_label = prediction["label"]
                gold = line["label"]
                mv = line["modal_verb"].lower()
                modal_verbs[mv]["total"] += 1
                labels[gold]["total"] += 1
                if gold != predicted_label:
                    modal_verbs[mv]["errors"] += 1
                    labels[gold]["errors"] += 1
                predictions.append(prediction["probs"])
                golds.append(labels_golds[gold])
            except RuntimeError:
                sent, l = line["sentence"], line["label"]
                warnings.warn(f"sentence too short: {sent}, {l}")

        print(accuracy(np.array(predictions), np.array(golds)))
    return  modal_verbs, labels

In [12]:
def accuracy(predictions, labels):
    return 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0]

In [16]:
archive = load_archive("../../models/gme_all/model.tar.gz", cuda_device=1)  
model = archive.model
model.eval()

Did not use initialization regex that was passed: .*linear_layers.*weight


BasicClassifier(
  (_text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_seq2vec_encoder): CnnEncoder(
    (_activation): ReLU()
    (conv_layer_0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (conv_layer_1): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (conv_layer_2): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
  )
  (_classification_layer): Linear(in_features=300, out_features=3, bias=True)
  (_loss): CrossEntropyLoss()
)

In [21]:
archive = load_archive("../../models/epos_balanced/model.tar.gz", cuda_device=1)  
model = archive.model
model.eval()

Did not use initialization regex that was passed: .*linear_layers.*weight


BasicClassifier(
  (_text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_seq2vec_encoder): CnnEncoder(
    (_activation): ReLU()
    (conv_layer_0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (conv_layer_1): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (conv_layer_2): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
  )
  (_classification_layer): Linear(in_features=300, out_features=3, bias=True)
  (_loss): CrossEntropyLoss()
)

In [22]:
predictor = ModalityPredictor(model, dataset_reader=ModalityDatasetReader())

In [18]:
modal_targets, labels = collect_errors_and_accuarcy("../../data/GME/test_modal-BIOSE-coarse.jsonl")


69.40298507462687


In [23]:
modal_targets, labels = collect_errors_and_accuarcy("../../data/EPOS_E/test_EPOS+MPQA_re-balanced.jsonl")

/home/nlp/shovalsa/anaconda3/envs/modalvenv/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: sentence too short: must be really expensive, ep
/home/nlp/shovalsa/anaconda3/envs/modalvenv/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: sentence too short: Maybe you can ., dy
/home/nlp/shovalsa/anaconda3/envs/modalvenv/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: sentence too short: I may now ., de


13.573407202216066


In [25]:
modal_targets

{'could': {'total': 38, 'errors': 18},
 'can': {'total': 54, 'errors': 45},
 'shall': {'total': 3, 'errors': 2},
 'must': {'total': 116, 'errors': 15},
 'should': {'total': 28, 'errors': 3},
 'may': {'total': 122, 'errors': 10}}